In [ ]:
#janomeインストール
!pip install janome

In [ ]:
#Tokenizerインスタンス作成
from janome.tokenizer import Tokenizer
tokenizer = Tokenizer()

In [ ]:
#解析するテキスト準備（手入力編）
text = """
　こんにちは、初めまして何とかかんとか生きてます。python大好きです。
"""

In [ ]:
#解析するテキスト準備（hagakureのテキストcsv読み込み編）
with open("/content/c_python.csv","r",encoding = "cp932") as f:
  lines = f.readlines()
  text =""
  for line in lines:
    text = text + line.split(",")[0]
  print(text)

In [ ]:
#janomeで分かち書き（単語に分割）
"""
surface（表層形）
part_of_speech（品詞）
infl_type（活用型）
infl_form（活用形）
base_form（基本形、見出し語）
reading（読み）
phonetic（発音）
"""

tokens = tokenizer.tokenize(text)
for token in tokens:
  print(token)
  # print(token.surface)
  # print(token.part_of_speech)
  # print(token.infl_type)
  # print(token.infl_form)
  # print(token.base_form)
  # print(token.reading)
  # print(token.phonetic)


In [ ]:
#ユーザー辞書作成(分割されてしまう固有名詞等を登録)
word_list = ["青空文庫","自然言語処理"]
userdict_str = ""
for word in word_list:
  userdict_one_str = word + ",-1,-1,-5000,名詞,一般,*,*,*,*,"+ word + ",*,*"
  userdict_str += userdict_one_str + "\n"
with open("userdic.csv","w",encoding="utf-8") as f:
     f.write(userdict_str)

In [ ]:
#janomeで分かち書き２（ユーザー辞書を持たせて）
tokenizer = Tokenizer("userdic.csv",udic_enc="utf-8")
tokens = tokenizer.tokenize(text)
for token in tokens:
  print(token)
  # print(token.surface)
  # print(token.part_of_speech)
  # print(token.infl_type)
  # print(token.infl_form)
  # print(token.base_form)
  # print(token.reading)
  # print(token.phonetic)

In [ ]:
#ワードクラウド作成の為に、半角スペース区切りの文字列作成

#ワードクラウドに入れたくない文字列はomit_listに登録
omit_list =["の","さん","こと","ん","そう"]
#英数字は除外して登録するが、例外的に登録したい文字列はisascii_listに登録
isascii_list = ["Python","google","https"]
nouns = ""
tokens = tokenizer.tokenize(text)

for token in tokens:
  if token.part_of_speech.split(",")[0] == "名詞":
    noun = token.base_form.replace("_","")
    if noun not in omit_list:
      if noun in isascii_list:
        nouns = nouns + noun + " "
      elif noun.isascii() == False:
        nouns = nouns + noun + " "
print(nouns)

In [ ]:
#各名詞の登場回数を調べる。（Atcoder的なやつ。やってみよう！！！）
from collections import Counter
nouns_list = nouns.split(" ")
counter = Counter(nouns_list)
counter.most_common()


In [ ]:
#ワードクラウドライブラリのインストール
!pip install wordcloud

In [ ]:
#インストールされているフォントの一覧を取得（デフォルトでは日本語のフォントはない）
!fc-list

In [ ]:
#手頃な日本語が使えるフリーフォントとしてipaexfontをインストール
!apt install fonts-ipaexfont
#他入れられる日本語フォントまとめ→　https://neos21.net/blog/2020/04/13-03.html

In [ ]:
#フォント追加確認
!fc-list

In [ ]:

#ワードクラウド作成、出力
from wordcloud import WordCloud
FONT_PATH ="/usr/share/fonts/opentype/ipaexfont-gothic/ipaexg.ttf"
wordcloud = WordCloud(font_path=FONT_PATH,
                      min_font_size=15,
                      max_font_size=250,
                      width=1000,
                      height=600,
                      background_color='orange',
                      colormap="autumn",
                      collocations =False,#重複をなくすおまじない
                      ).generate(nouns)
wordcloud.to_file("result.png")

#colormap一覧→　https://karupoimou.hatenablog.com/entry/2019/05/17/153207

In [ ]:
#透過pngの画像を用意して好きな形のワードクラウドを作成する
#透過pngを作成できるサイト→https://www.peko-step.com/tool/alphachannel.html
#画像を読み込んでマスク作成。
import cv2
 
IMG_PATH = "/content/mask.png"
 
img = cv2.imread(IMG_PATH, -1)
a_img = img[:, :, 3]
result_img = cv2.bitwise_not(a_img)
# cv2.imwrite('mask.png', result_img)

In [ ]:
#引数にmaskを与えて作成。
from wordcloud import WordCloud
FONT_PATH ="/usr/share/fonts/opentype/ipaexfont-gothic/ipaexg.ttf"
wordcloud = WordCloud(font_path=FONT_PATH,
                      min_font_size=5,
                      max_font_size=150,
                      width=600,
                      height=600,
                      background_color='pink',
                      colormap="autumn",
                      collocations =False,
                      mask=result_img,
                      ).generate(nouns)
wordcloud.to_file("result.png")